In [ ]:
# In this notebook, we will evaluate the results of the ellipse dataset training runs


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import json
from scipy.spatial.distance import directed_hausdorff

from sys import path
path.append('../../..')

base_output_dir = pathlib.Path('../../../output/lassen')
files = list(base_output_dir.glob('interfacial_ae_v5_*'))
len(files)


In [ ]:
from src.interface_representation.interface_transformations import InterfaceRepresentationType

In [ ]:
def read_loss_curve_from_file(outdir):
    loss_path = outdir / 'loss_history.json'
    with open(loss_path) as f:
        loss_curve = json.load(f)
    return loss_curve

def load_preds(outdir):
    pred_path = outdir / 'preds_75.npz'
    preds = np.load(pred_path)['preds']
    return preds

def load_gt(outdir):
    gt_path = outdir / 'gts_75.npz'
    gt = np.load(gt_path)['gts']
    return gt

def load_interface_type(filename):
    final_part = '_'.join(filename.stem.split('_')[-2:])
    
    str_to_type = {
        '00_datadirtanh': InterfaceRepresentationType.TANH,
        'datadirexact_sdf': InterfaceRepresentationType.SDF_EXACT,
        'datadirapprox_sdf': InterfaceRepresentationType.SDF_APPROX,
        'datadirtanh_sharp': InterfaceRepresentationType.TANH,
        'datadirtanh_smooth': InterfaceRepresentationType.TANH
    }
    
    return str_to_type[final_part]

def load_epsilon(filename):
    final_part = '_'.join(filename.stem.split('_')[-2:])
    
    str_to_epsilon = {
        '00_datadirtanh': 1/64,
        'datadirexact_sdf': None,
        'datadirapprox_sdf': None,
        'datadirtanh_sharp': 1/128,
        'datadirtanh_smooth': 1/32,
    }
    
    return str_to_epsilon[final_part]


def filename_to_plot_label(filename):
    final_part = '_'.join(filename.stem.split('_')[-2:])
    
    str_to_label = {
        '00_datadirtanh': 'Tanh 1/64',
        'datadirexact_sdf': 'Exact SDF',
        'datadirapprox_sdf': 'Approx. SDF',
        'datadirtanh_sharp': 'Tanh 1/128',
        'datadirtanh_smooth': 'Tanh 1/32'
    }
    
    return str_to_label[final_part]


def compute_phi_sharp_from_tanh(phi):
    return np.heaviside(phi - 0.5, 1)


def compute_phi_sharp_from_sdf(psi):
    return np.heaviside(-psi, 1)


def dice_coefficient(gt_patch, pred_patch, level: float = 0.5):
    """Returns the dice coefficient of foreground region, obtained by thresholding the images at level
    """
    gt_patch = gt_patch > level
    pred_patch = pred_patch > level
    intersection = np.sum(gt_patch * pred_patch)
    union = np.sum(gt_patch) + np.sum(pred_patch)
    return 2 * intersection / union


def hausdorff_distance(gt_patch, pred_patch, level: float = 0.5):
    """Returns the Hausdorff distance of the foreground region, obtained by thresholding the images at level

    Note:
        The distance is in units of voxels, assumes isotropic voxels

    Args:
        gt_patch: Ground truth patch
        pred_patch: Predicted patch
        level: Threshold level
        max_num_points: Maximum number of points to use in the distance calculation (for speed purposes)
    """
    gt_patch = gt_patch > level
    pred_patch = pred_patch > level

    gt_indices = np.argwhere(gt_patch) * 1 / 64
    pred_indices = np.argwhere(pred_patch) * 1 / 64

    if len(gt_indices) == 0 or len(pred_indices) == 0:
        return np.nan

    h_1 = directed_hausdorff(gt_indices, pred_indices)[0]
    h_2 = directed_hausdorff(pred_indices, gt_indices)[0]
    return max(h_1, h_2)


def get_phi_sharp_pred_and_gt(pred, gt, interface_type):
    if interface_type == InterfaceRepresentationType.TANH:
        pred = compute_phi_sharp_from_tanh(pred)
        gt = compute_phi_sharp_from_tanh(gt)
    elif interface_type == InterfaceRepresentationType.SDF_EXACT:
        pred = compute_phi_sharp_from_sdf(pred)
        gt = compute_phi_sharp_from_sdf(gt)
    elif interface_type == InterfaceRepresentationType.SDF_APPROX:
        pred = compute_phi_sharp_from_sdf(pred)
        gt = compute_phi_sharp_from_sdf(gt)
    else:
        raise ValueError('Unknown interface type')
    return pred, gt
    

In [ ]:
files

In [ ]:
from tqdm import tqdm

plot_label_to_data = {}

for file in files:
    preds = load_preds(file)
    gts = load_gt(file)
    
    interfacetype = load_interface_type(file)
    plot_label = filename_to_plot_label(file)
    epsilon = load_epsilon(file)
    
    print(interfacetype, epsilon)
    
    assert len(preds) == len(gts)
    
    means = []
    dices = []
    hds = []
    
    for pred, gt in zip(preds, gts):
        pred, gt = get_phi_sharp_pred_and_gt(pred, gt, interfacetype)
        
        mse = np.mean((pred - gt)**2)
        means.append(mse)
        
        dice = dice_coefficient(gt, pred)
        dices.append(dice)
        
        hd = hausdorff_distance(gt, pred)
        hds.append(hd)
        
    plot_label_to_data[plot_label] = {
        'MSE': means,
        'Dice': dices,
        'Hausdorff': hds
    }
        
    

In [ ]:
# For each metric, create a set of box plots with interface type on the x-axis
    
# For each metric, create a set of box plots with interface type on the x-axis

metrics = ['MSE', 'Dice', 'Hausdorff']  # Add 'Hausdorff' if needed
higher_or_lower = ['lower', 'higher', 'lower']
fig, axs = plt.subplots(1, len(metrics), figsize=(18, 6))

for i, metric in enumerate(metrics):
    data = [plot_label_to_data[label][metric] for label in plot_label_to_data]
    labels = list(plot_label_to_data.keys())
    axs[i].boxplot(data, tick_labels=labels)
    axs[i].set_title(f'{metric} ({higher_or_lower[i]} better)')
    axs[i].set_ylabel(metric)
    axs[i].spines['top'].set_visible(False)
    axs[i].spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Compare droplet PDFs for gt/pred, for each interface type
from scipy.ndimage import label

def compute_droplet_pdf(arr):
    # Assume that arr is binary. Label connected components, and return the histogram of the sizes
    labeled_arr, num_labels = label(arr)
    sizes = np.bincount(labeled_arr.ravel())
    return sizes[1:]

plot_label_to_data = {}

for file in files:
    preds = load_preds(file)
    gts = load_gt(file)
    
    interfacetype = load_interface_type(file)
    plot_label = filename_to_plot_label(file)
    epsilon = load_epsilon(file)
    
    print(interfacetype, epsilon)
    
    assert len(preds) == len(gts)
    
    pred_sizes = []
    gt_sizes = []
    
    for pred, gt in zip(preds, gts):
        pred, gt = get_phi_sharp_pred_and_gt(pred, gt, interfacetype)
        
        pred_size = compute_droplet_pdf(pred)
        gt_size = compute_droplet_pdf(gt)
        
        pred_sizes.extend(pred_size)
        gt_sizes.extend(gt_size)
        
    plot_label_to_data[plot_label] = {
        'gt': gt_sizes,
        'pred': pred_sizes
    }
        
        

In [ ]:

# Plot the droplet PDFs
fig, axs = plt.subplots(1, len(plot_label_to_data), figsize=(18, 6))

for i, label in enumerate(plot_label_to_data):
    data = plot_label_to_data[label]
    axs[i].hist(data['gt'], bins=50, alpha=0.5, label='GT')
    axs[i].hist(data['pred'], bins=50, alpha=0.5, label='Pred')
    axs[i].set_title(label)
    axs[i].set_xlabel('Droplet size')
    axs[i].set_ylabel('Frequency')
    axs[i].legend()
    axs[i].spines['top'].set_visible(False)
    axs[i].spines['right'].set_visible(False)


# Summary

- The performance metrics show generally better results for the exact SDF, worse for the approximate SDF, and no significant difference between the tanh interfaces. But all have very high reconstruction accuracy.
- The comparison of droplet size PDFs is not meaningful on the synthetic data, as there is only 1 droplet per volume

# Next steps

- Repeat each run for 3 random seeds to get uncertainty estimates in metrics due to training process
- Add heaviside interface representation
- Try a new synthetic dataset comprised of a varying number (e.g. 1-10) of spheres with random radii and positions, as the ellipse dataset is too simple 
- Sample droplet radii from log-normal distribution, which is more representative of atomisation? Choose random max number of droplets (40) and then sample random radius and centre, and accept if no overlap with existing droplets.
- Stick with pure AE, but future work with ROM would want to use beta-VAE.
- Compare activation functions for the AE
- Can we look at network depth vs. performance?
- Push to 4 downsamples and 4 upsamples, 

To be more concrete, we will:
- Create a V8 dataset, where the droplet radii are log-normally distributed and there is no overlap
- Re-create the V5 dataset with half the number of samples, change epochs to 100, and add two new types of interface representation

Tonight: run + upload V5, upload V7, create V8 dataset